# La notion de Class

Une classe est un ensemble de code contenant des variables et des fonctions permettant de créer des objets.
Une classe peut contenir plusieurs objets.
Une classe définit des attributs et des méthodes.
Par exemple, imaginons une classe `Voiture` qui servira à créer des objets qui sont des voitures.
Cette classe va pouvoir définir un attribut couleur, un attribut vitesse, etc.
Ces attributs correspondent à des propriétés qui peuvent exister pour une voiture.
La classe `Voiture` pourra également définir une méthode `rouler()`. Une méthode correspond en quelque sorte à une action, ici l’action de rouler peut être réalisée pour une voiture.
![Class](/Users/dpicca/PycharmProjects/POOP/img/classe_ex.png)

## La classe Position

Pour calculer la densité de population, nous devons connaître la position d'un agent sur une carte.
Or pour le moment il ne connaît que le concept `Agent`(le pion rouge de notre jeu de touché/coulé).

La position est données par la longitude et la latitude de l'agent dans le monde

<img src="/Users/dpicca/PycharmProjects/POOP/img/lat_long.png" width="800"/>

Créons donc une nouvelle classe qui nous servira à calculer les positions à la fois des Agents et des coins de nos futures zones.
Si nous voulons accéder à la longitude, nous devons écrire `position.longitude()`.

Or nous voulons plutôt écrire `position.longitude`, afin que la longitude soit utilisée comme un attribut et non comme une méthode !
Cela peut vous paraître anodin mais, croyez-moi, c'est important ! En effet, vous ne voulez pas que votre instance fasse une action, ce qui correspond plutôt à une méthode, mais qu'elle vous renvoie une valeur qui lui est propre. Nous utilisons donc une propriété.
Pour créer une propriété nous ajoutons, juste avant la définition de la méthode, le mot-clé `@property`

NB: Les coordonnées de chaque agent sont en degrés.
Or, en mathématiques, il est préférable d'utiliser des radians.
Il s'agit tout simplement d'une unité de mesure différente qui sera plus pratique à utiliser.
La longitude en radians est égale à sa longitude en degrés * pi divisé par 180. Soit le calcul suivant :

`longitude = longitude_degrees * pi / 180`

In [36]:
import math
class Position:
    def __init__(self, longitude_degrees, latitude_degrees):
        self.latitude_degrees = latitude_degrees
        self.longitude_degrees = longitude_degrees

    @property
    def longitude(self):
        # Longitude in radians
        return self.longitude_degrees * math.pi / 180

    @property
    def latitude(self):
        # Latitude in radians
        return self.latitude_degrees * math.pi / 180

Maintenant vous pouvez appeler les méthodes `longitude` et `latitude` comme des propriétés.

In [37]:
import json
def main():
    for agent_attributes in json.load(open("../../data/agents-100k.json")):
        latitude = agent_attributes.pop("latitude")
        longitude = agent_attributes.pop("longitude")
        position = Position(latitude, longitude)
        print(position.latitude)
        print(position.longitude)
        break

main()

1.7603112756100432
0.5786150090711938


## La classe Zone dans notre projet
Créons la classe Zone qui a trois attributs :
la position de son coin inférieur gauche, la position de son coin supérieur droit et ses habitants.
Lorsque nous créerons chaque zone, nous déterminerons sa position et disons que, par défaut, la zone est vide.

In [38]:
class Zone:
    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = 0

## Les attributs de classe
Si nous reprenons notre jeu de bataille navale, nous voyons que nous avons plusieurs lignes. Une ligne a une abscisse minimale, une abscisse maximale et un intervalle distinguant chaque zone minimale de jeu.
Autrement dit, toujours dans le jeu, nous avons 9 zones comprises entre 0 et 9 avec un intervalle de 1 pour cette première ligne.

Nous allons utiliser la même logique dans notre grille mais pas les mêmes valeurs, puisque la longitude est comprise entre -180 et 180 degrés et que la latitude, elle, est comprise entre -90 et 90 degrés.
Pour créer la première ligne de notre grille, nous devons donc définir certaines variables qui, a priori, ne changeront pas.

Nous voulons qu'elles soient associées à la classe et non pas aux instances. En effet, si nous changeons l'abscisse minimale de nos zones, nous voulons le faire pour toutes les instances en une fois. Nous pouvons raisonnablement nous dire que le champ d'action de ces variables sera plus large que celui des attributs d'instance.

Nous appelons ces variables des `attributs de classe` car elles appartiennent à la classe dans son ensemble, et non à l'instance. C'est notre zone de manière générale qui a une latitude minimale et une latitude maximale, et non l'instance !

In [39]:
class Zone:
    ZONES=[]
    MIN_LONGITUDE_DEGREES = -180
    MAX_LONGITUDE_DEGREES = 180
    MIN_LATITUDE_DEGREES = -90
    MAX_LATITUDE_DEGREES = 90
    WIDTH_DEGREES = 1 # degrees of longitude
    HEIGHT_DEGREES = 1 # degrees of latitude

    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = 0

    def initialize_zones(self):
        for latitude in range (self.MIN_LATITUDE_DEGREES, self.MAX_LATITUDE_DEGREES, self.HEIGHT_DEGREES):
            for longitude in range(self.MIN_LONGITUDE_DEGREES, self.MAX_LONGITUDE_DEGREES, self.WIDTH_DEGREES):
                bottom_left_corner = Position(latitude, longitude)
                top_right_corner = Position(longitude + self.WIDTH_DEGREES, latitude + self.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                self.ZONES.append(zone)
        print(len(self.ZONES))

### Decorators et méthodes privées

Mais... Comment lancer une méthode sur une instance alors que cette même méthode est celle qui, justement, est censée les créer ?

Il faudrait une méthode qui soit globale, au niveau de la classe, et non de l'instance.
Alors, comment le faire pour les méthodes ? Vous ajoutez  `@classmethod`  juste avant.

ATTENTION: Bien. Étant donné que nous ne sommes plus au niveau de l'instance mais au niveau de la classe, nous allons remplacer `self par `cls` (afin de ne pas confondre).

In [40]:
class Zone:
    ZONES=[]
    MIN_LONGITUDE_DEGREES = -180
    MAX_LONGITUDE_DEGREES = 180
    MIN_LATITUDE_DEGREES = -90
    MAX_LATITUDE_DEGREES = 90
    WIDTH_DEGREES = 1 # degrees of longitude
    HEIGHT_DEGREES = 1 # degrees of latitude

    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = 0
    @classmethod
    def initialize_zones(cls):
        for latitude in range (cls.MIN_LATITUDE_DEGREES, cls.MAX_LATITUDE_DEGREES, cls.HEIGHT_DEGREES):
            for longitude in range(cls.MIN_LONGITUDE_DEGREES, cls.MAX_LONGITUDE_DEGREES, cls.WIDTH_DEGREES):
                bottom_left_corner = Position(longitude, latitude)
                top_right_corner = Position(longitude + cls.WIDTH_DEGREES, latitude + cls.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                cls.ZONES.append(zone)
Zone.initialize_zones()

# Ajoutons les habitants
Nous avons la grille, les zones, les positions et les agents.
Mais pour l'instant les habitants sont hors des zones, un peu comme s'ils lévitaient...
Faisons-les redescendre sur terre en les plaçant sur la grille !
## Trouver la zone d'habitation

Nous allons commencer par créer une méthode qui trouve dans quelle zone "habite" l'agent.
Puis nous mettrons à jour la zone pour calculer la population !

Premièrement, trouvons les zones dans lesquelles habitent chacun de nos agents. Étant donné que nous allons parcourir les zones, nous créons une méthode de classe.
La solution est de trouver l'index de la zone à partir de l'index de la position.

In [41]:
class Zone:

    ZONES = []
    MIN_LONGITUDE_DEGREES = -180
    MAX_LONGITUDE_DEGREES = 180
    MIN_LATITUDE_DEGREES = -90
    MAX_LATITUDE_DEGREES = 90
    WIDTH_DEGREES = 1 # degrees of longitude
    HEIGHT_DEGREES = 1 # degrees of latitude

    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = []

    def contains(self, position):
        return position.longitude >= min(self.corner1.longitude, self.corner2.longitude) \
               and position.longitude < max(self.corner1.longitude, self.corner2.longitude) \
               and position.latitude >= min(self.corner1.latitude, self.corner2.latitude) \
               and position.latitude < max(self.corner1.latitude, self.corner2.latitude)

    @classmethod
    def find_zone_that_contains(cls, position):
        # Compute the index in the ZONES array that contains the given position
        longitude_index = int((position.longitude_degrees - cls.MIN_LONGITUDE_DEGREES)/ cls.WIDTH_DEGREES)
        latitude_index = int((position.latitude_degrees - cls.MIN_LATITUDE_DEGREES)/ cls.HEIGHT_DEGREES)
        longitude_bins = int((cls.MAX_LONGITUDE_DEGREES - cls.MIN_LONGITUDE_DEGREES) / cls.WIDTH_DEGREES) # 180-(-180) / 1
        zone_index = latitude_index * longitude_bins + longitude_index

        # Just checking that the index is correct
        zone = cls.ZONES[zone_index]
        assert zone.contains(position)

        return zone

    @classmethod
    def initialize_zones(cls):
        # Note that this method is "private": we prefix the method name with "_".
        cls.ZONES = []
        for latitude in range(cls.MIN_LATITUDE_DEGREES, cls.MAX_LATITUDE_DEGREES, cls.HEIGHT_DEGREES):
            for longitude in range(cls.MIN_LONGITUDE_DEGREES, cls.MAX_LONGITUDE_DEGREES, cls.WIDTH_DEGREES):
                bottom_left_corner = Position(longitude, latitude)
                top_right_corner = Position(longitude + cls.WIDTH_DEGREES, latitude + cls.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                cls.ZONES.append(zone)




On modifie la classe `Agent` par consequent

In [42]:
class Agent:

    def __init__(self, position, **agent_attributes):
        self.position = position
        for attr_name, attr_value in agent_attributes.items():
            setattr(self, attr_name, attr_value)



def main():
    Zone.initialize_zones()
    for agent_attributes in json.load(open("../../data/agents-100k.json")):
        latitude = agent_attributes.pop("latitude")
        longitude = agent_attributes.pop("longitude")
        position = Position(longitude, latitude)
        agent = Agent(position, **agent_attributes)
        zone = Zone.find_zone_that_contains(position)

main()

## Ajoutez un habitant !

Commençons par ajouter une méthode d'instance  `add_inhabitant(agent)`  :

    def main():

        for agent_attributes in json.load(open("agents-100k.json")):
            ...
            zone.add_inhabitant(agent)

Puis créons la méthode associée dans la classe. Cette méthode va ajouter l'agent à une liste. Il nous faut donc également mettre à jour la méthode __init__() pour que les habitants par défaut ne soient pas 0 mais une liste vide :

In [43]:
class Zone:

    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = []

    def add_inhabitant(self):
        pass

Comment ajouter un élément à une liste ? En utilisant la méthode append() !

In [44]:
class Zone:
    ...
    def add_inhabitant(self, inhabitant):
        self.inhabitants.append(inhabitant)

Dernière ligne droite à présent ! Nous souhaitons connaître la population d'une zone, c'est-à-dire le nombre total d'habitants, que nous diviserons par le nombre de kilomètres carrés afin d'obtenir la densité de population. Nous allons donc créer une nouvelle méthode qui renvoie le nombre total d'éléments dans la liste de population.

In [45]:
class Zone:
    def population(self):
        return len(self.inhabitants)

Étant donné que cette méthode existe uniquement pour me donner une information, et non vraiment pour effectuer une action, je vais la transformer en propriété :

In [46]:
class Zone:
    ...
    @property
    def population(self):
        return len(self.inhabitants)

Testons ! J'ajoute  `print(zone.population)`  dans la fonction  `main()` pour vérifier que les habitants sont bien ajoutés.

In [51]:
import json
import math

class Agent:

    def __init__(self, position, **agent_attributes):
        self.position = position
        for attr_name, attr_value in agent_attributes.items():
            setattr(self, attr_name, attr_value)

class Position:
    def __init__(self, longitude_degrees, latitude_degrees):
        self.latitude_degrees = latitude_degrees
        self.longitude_degrees = longitude_degrees

    @property
    def longitude(self):
        # Longitude in radians
        return self.longitude_degrees * math.pi / 180

    @property
    def latitude(self):
        # Latitude in radians
        return self.latitude_degrees * math.pi / 180

class Zone:

    ZONES = []
    MIN_LONGITUDE_DEGREES = -180
    MAX_LONGITUDE_DEGREES = 180
    MIN_LATITUDE_DEGREES = -90
    MAX_LATITUDE_DEGREES = 90
    WIDTH_DEGREES = 1 # degrees of longitude
    HEIGHT_DEGREES = 1 # degrees of latitude

    def __init__(self, corner1, corner2):
        self.corner1 = corner1
        self.corner2 = corner2
        self.inhabitants = []

    @property
    def population(self):
        return len(self.inhabitants)

    def add_inhabitant(self, inhabitant):
        self.inhabitants.append(inhabitant)

    def contains(self, position):
        return position.longitude >= min(self.corner1.longitude, self.corner2.longitude) \
               and position.longitude < max(self.corner1.longitude, self.corner2.longitude) \
               and position.latitude >= min(self.corner1.latitude, self.corner2.latitude) \
               and position.latitude < max(self.corner1.latitude, self.corner2.latitude)

    @classmethod
    def find_zone_that_contains(cls, position):
        # Compute the index in the ZONES array that contains the given position
        longitude_index = int((position.longitude_degrees - cls.MIN_LONGITUDE_DEGREES)/ cls.WIDTH_DEGREES)
        latitude_index = int((position.latitude_degrees - cls.MIN_LATITUDE_DEGREES)/ cls.HEIGHT_DEGREES)
        longitude_bins = int((cls.MAX_LONGITUDE_DEGREES - cls.MIN_LONGITUDE_DEGREES) / cls.WIDTH_DEGREES) # 180-(-180) / 1
        zone_index = latitude_index * longitude_bins + longitude_index

        # Just checking that the index is correct
        zone = cls.ZONES[zone_index]
        assert zone.contains(position)

        return zone

    @classmethod
    def initialize_zones(cls):
        # Note that this method is "private": we prefix the method name with "_".
        cls.ZONES = []
        for latitude in range(cls.MIN_LATITUDE_DEGREES, cls.MAX_LATITUDE_DEGREES, cls.HEIGHT_DEGREES):
            for longitude in range(cls.MIN_LONGITUDE_DEGREES, cls.MAX_LONGITUDE_DEGREES, cls.WIDTH_DEGREES):
                bottom_left_corner = Position(longitude, latitude)
                top_right_corner = Position(longitude + cls.WIDTH_DEGREES, latitude + cls.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                cls.ZONES.append(zone)




def main():
    Zone.initialize_zones()
    for agent_attributes in json.load(open("../../data/agents-100k.json")):
        latitude = agent_attributes.pop("latitude")
        longitude = agent_attributes.pop("longitude")
        position = Position(longitude, latitude)
        agent = Agent(position, **agent_attributes)
        zone = Zone.find_zone_that_contains(position)
        zone.add_inhabitant(agent)
        print(zone.population)

main()


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
3
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
1
2
1
1
1
3
2
1
1
1
1
2
1
1
3
1
1
2
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
2
1
1
4
1
3
1
1
1
1
1
1
2
1
2
1
2
1
1
1
1
1
1
1
2
1
1
1
2
2
1
2
1
1
1
2
1
1
1
1
2
1
1
1
1
1
2
1
1
2
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
5
2
1
1
1
1
1
3
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
2
1
1
1
1
2
3
1
2
1
1
1
1
1
2
3
2
1
1
1
2
1
1
1
1
1
2
1
2
2
2
2
3
1
1
1
1
1
1
1
3
1
1
1
2
1
1
2
1
2
1
3
1
1
2
1
2
2
3
1
1
1
1
1
1
1
1
1
2
1
2
4
2
2
1
1
1
2
2
1
1
1
2
1
1
1
2
1
2
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
4
1
2
2
1
1
2
2
1
1
1
4
1
1
1
2
1
3
2
1
1
1
1
1
4
2
1
1
1
1


KeyboardInterrupt: 

# Encapsulation (ou méthodes privées)
Une classe est comme une capsule :
elle contient en elle-même tout ce dont elle a besoin pour fonctionner.
Lorsque nous l'utilisons, une partie de son fonctionnement nous est caché afin de ne nous donner accès qu'aux fonctions essentielles pour nous.

Certains langages sont très stricts au sujet de l'encapsulation.
Vous ne pouvez vraiment pas accéder à des méthodes privées en dehors de la classe.
Python est plus laxiste.
La philosophie du langage est plus axée sur la confiance : si nous voulons accéder à un élément protégé ou privé c'est que nous avons une bonne raison de le faire.

C'est pourquoi Python utilise cette notion d'encapsulation et d'éléments privés mais à travers une convention de syntaxe. Tout est public en Python, néanmoins il vous sera plus difficile d'accéder à des éléments considérés comme privés.

## L'encapsulation appliquée à notre programme

In [ ]:
class Zone
    ...

    @classmethod
    def _initialize_zones(cls):
        for latitude in range (cls.MIN_LATITUDE_DEGREES, cls.MAX_LATITUDE_DEGREES, cls.HEIGHT_DEGREES):
            for longitude in range(cls.MIN_LONGITUDE_DEGREES, cls.MAX_LONGITUDE_DEGREES, cls.WIDTH_DEGREES):
                bottom_left_corner = Position(longitude, latitude)
                top_right_corner = Position(longitude + cls.WIDTH_DEGREES, latitude + cls.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                cls.ZONES.append(zone)

def main():
    for agent_attributes in json.load(open("agents-100k.json")):
        latitude = agent_attributes.pop("latitude")
        longitude = agent_attributes.pop("longitude")
        position = Position(longitude, latitude)
        agent = Agent(position, **agent_attributes)
        zone = Zone.find_zone_that_contains(position)

L'initialisation des zones devrait se faire de l'intérieur de la classe, et non de l'extérieur. Néanmoins, je dois pouvoir quand même le faire en cas de besoin. Je vais donc "protéger" la méthode de la classe `initialize_zones(cls)` et l'enlever de notre fonction `main().

Après tout, nous n'avons pas envie de savoir s'il faut initialiser la grille ou pas hors de la zone. C'est une vérification qui devrait être faite à l'intérieur. Étant donné que nous exécutons la fonction `find_zone_that_contains(position)`, je vais vérifier si les zones existent déjà au tout début :

In [ ]:
class Zone
    ...

    @classmethod
    def _initialize_zones(cls):
        for latitude in range (cls.MIN_LATITUDE_DEGREES, cls.MAX_LATITUDE_DEGREES, cls.HEIGHT_DEGREES):
            for longitude in range(cls.MIN_LONGITUDE_DEGREES, cls.MAX_LONGITUDE_DEGREES, cls.WIDTH_DEGREES):
                bottom_left_corner = Position(longitude, latitude)
                top_right_corner = Position(longitude + cls.WIDTH_DEGREES, latitude + cls.HEIGHT_DEGREES)
                zone = Zone(bottom_left_corner, top_right_corner)
                cls.ZONES.append(zone)

def main():
    for agent_attributes in json.load(open("agents-100k.json")):
        latitude = agent_attributes.pop("latitude")
        longitude = agent_attributes.pop("longitude")
        position = Position(longitude, latitude)
        agent = Agent(position, **agent_attributes)
        zone = Zone.find_zone_that_contains(position)

# Exercises sur le decorators et encapsulation (avec pandas)